In [0]:
#@title <font color='Blue'>**Overheads**</font>

# Author: Yu-Man Tam
# Last updated: 1/23/2020

# Reference: Deep Hedging (2019, Quantitative Finance) by Buehler et al.
# https://www.tandfonline.com/doi/abs/10.1080/14697688.2019.1571683

from tqdm import trange, tqdm
from tqdm.keras import TqdmCallback
import pathos.multiprocessing as mp
from IPython.display import Math, HTML, clear_output

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload

import numpy as np
import QuantLib as ql
import tensorflow as tf
from itertools import repeat
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
                                            ReduceLROnPlateau
from tensorflow.compat.v1.keras.optimizers import Adam
from tensorflow.keras.models import Model
import tensorflow_addons as tfa

# Configure Tensorflow
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

# Disable warnings for presentation purpose
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, os.getcwd() + "/lib")

from Stochastic_Processes import BlackScholesProcess
from Utilities import train_test_split, reshape_1D
from Deep_Hedging import Deep_Hedging
from Validation import Validation

clear_output()
print("\nFinish installing and importing all necessary libraries!")

**Import all neccessary python, quantitative finance, and machine learning software libraries.**

In [0]:
#@title <font color='Blue'>**User Inputs**</font>

# Geometric Brownian Motion.
N = 30 # Number of time steps (in days)

S0 = 100.0 # Stock price at time = 0
sigma = 0.2 # Implied volatility
risk_free = 0.0 # Risk-free rate
dividend = 0.0 # Continuous dividend yield

Ktrain = 1*(10**5) # Size of training sample.
Ktest_ratio = 0.2 # Fraction of training sample as testing sample.

# European call option (short).
strike = S0
payoff_func = lambda x: -np.maximum(x - strike, 0.0)
calculation_date = ql.Date.todaysDate()
maturity_date = ql.Date.todaysDate() + N
num_days_in_a_year = 365 # Actual365Fixed() is currently implemented

# Proportional transaction cost.
epsilon = np.power(2.0,-11)*0.0

# Neural network (NN) structure
m = 25 # Number of neurons in each hidden layer.
d = 3 # Number of hidden layers (Note including input nor output layer)                                         

# Information set (in string)
# Choose from: S, log_S
information_set = "log_S"

# Loss function
# loss_type = "CVaR" (Expected Shortfall) -> loss_param = alpha 
# loss_type = "Entropy" -> loss_param = lambda
loss_type = "Entropy"
loss_param = 1.0

# Other NN parameters
lr = 5e-3 # Learning rate
batch_size=256 # Batch size
epochs=1000 # Number of epochs

print("Finish loading user inputs!")

**Provide input parameters for Monte Carlo simulation, call option, transaction cost, loss function, and deep hedging algorithm.**

In [0]:
#@title <font color='Blue'>**Monte Carlo Simulation - Generate Random Paths of Stock Prices**</font>
stochastic_process = BlackScholesProcess(s0 = S0, sigma = sigma, \
                      risk_free = risk_free, dividend = dividend)
maturity = N/365

try:
  print("\nCheck if there is usable data from previous simulation...")
  S = np.load("./data/stock_" + str(int(np.log10(Ktrain))) + ".npy")
  print("Successfully loaded data!!!\n")
except:
  S = stochastic_process.gen_path(maturity, N, int(Ktrain/(1-Ktest_ratio)))
  np.save("./data/stock_" + str(int(np.log10(Ktrain))) + ".npy", S)

print("s0 = " + str(S0))
print("sigma = " + str(sigma))
print("risk_free = " + str(risk_free) + "\n")
print("Number of time steps = " + str(N))
print("Length of each time step = " + "1/365\n")
print("Simulation Done!")

In [0]:
#@title <font color='Blue'>**Prepare data to be fed into the deep hedging algorithm.**</font>

payoff_T = payoff_func(S[:,-1]) # Payoff of the call option

trade_set =  np.stack((S),axis=1) # Trading set

if information_set is "S":
  I =  np.stack((S),axis=1) # Information set
elif information_set is "log_S":
  I =  np.stack((np.log(S)),axis=1) # Information set

# Structure of xtrain:
#   1) Certainty equivalent: w (dim = 1)
#   2) Trade set: [S]
#   2) Information set: [S] 
#   3) payoff (dim = 1)
xtrain = [np.zeros(len(S))]
for i in range(N+1):
    xtrain += [trade_set[i,:]]
    if i != N:
        xtrain += [I[i,:]]
xtrain += [payoff_T]

# Split the entire sample into a training sample and a testing sample.
[xtrain, xtest] = train_test_split(xtrain, test_size=Ktest_ratio)
[S_train, S_test] = train_test_split([S], test_size=Ktest_ratio)
[option_payoff_train, option_payoff_test] = \
    train_test_split([payoff_T], test_size=Ktest_ratio)

print("Finish preparing data!")

In [0]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Simple Network)!**</font>
%autoreload 2
strategy_type = "simple"
optimizer = tfa.optimizers.LazyAdam(learning_rate=lr)

# Setup and compile the model
model_simple = Deep_Hedging(N=N, d=d+2, m=m, risk_free=risk_free, \
                      maturity=maturity).model( \
                      strategy_type=strategy_type, epsilon = epsilon, \
                      loss_type=loss_type, loss_param = loss_param)
model_simple.compile(optimizer=optimizer)

# Stopping criteria
model_data_dir ="./data/"
best_model_file = model_data_dir + \
          "model_" + strategy_type + "_"  + str(epsilon) + ".h5"

early_stopping = EarlyStopping(monitor="loss", \
          patience=100, min_delta=1e-4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="loss", \
          factor=0.95, patience=5, min_delta=1e-3, verbose=0)
model_checkpoint = ModelCheckpoint(best_model_file, \
          monitor="loss", save_best_only=True, \
          save_weights_only=False, verbose=0)

callbacks = [early_stopping, reduce_lr, model_checkpoint, TqdmCallback()]

# Try to preload model weights if possible.
# try:
#  model_simple.load_weights(best_model_file)
# except:
#  pass

# Fit the model.
model_fit = model_simple.fit(x=xtrain, batch_size=batch_size, epochs=epochs, \
            validation_data=(xtest, np.empty(0)), callbacks=callbacks, \
            verbose=0)
model_simple.save(best_model_file)
print("Finished running deep hedging algorithm! (Simple Network)")

In [0]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Recurrent Network)!**</font>
strategy_type = "recurrent"
optimizer = tfa.optimizers.LazyAdam(learning_rate=lr)

# Setup and compile the model
model_recurrent = Deep_Hedging(N=N, d=d+2, m=m, risk_free=risk_free, \
                      maturity=maturity).model( \
                      strategy_type=strategy_type, epsilon = epsilon, \
                      loss_type=loss_type, loss_param = loss_param)
model_recurrent.compile(optimizer=optimizer)

# Stopping criteria
model_data_dir ="./data/"
best_model_file = model_data_dir + \
          "model_" + strategy_type + "_"  + str(epsilon) + ".h5"

early_stopping = EarlyStopping(monitor="loss", \
          patience=20, min_delta=1e-4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="loss", \
          factor=0.95, patience=5, min_delta=1e-3, verbose=0)
model_checkpoint = ModelCheckpoint(best_model_file, \
          monitor="loss", save_best_only=True, \
          save_weights_only=False, verbose=0)

callbacks = [early_stopping, reduce_lr, model_checkpoint, TqdmCallback()]

# Try to preload model weights if possible.
try:
  model_recurrent.load_weights(best_model_file)
except:
  pass

# Fit the model.
model_fit = model_recurrent.fit(x=xtrain, batch_size=batch_size, epochs=epochs, \
            validation_data=(xtest, np.empty(0)), callbacks=callbacks, \
            verbose=0)
model_recurrent.save(best_model_file)
print("Finished running deep hedging algorithm! (Recurrent Network)")

In [0]:
#@title <font color='Blue'>**Results: Option Prices**</font>
report = Validation(model = model_simple, data = xtest, N = N, process = stochastic_process)

# Specify the instrument as an European call option
instrument = report.get_instrument(name = "European_Call", strike = strike, \
                                    maturity_date = maturity_date)

BS_price = report.get_model_PV(instrument)
risk_neutral_price = report.get_risk_neutral_PV()
nn_simple_price = model_simple.evaluate(xtest, batch_size=batch_size, verbose=0)
nn_recurrent_price = model_recurrent.evaluate(xtest, batch_size=batch_size, verbose=0)

print("The Black-Scholes model price is %2.3f." % BS_price)
print("The Risk Neutral price is %2.3f." % risk_neutral_price)
print("The Deep Hedging (with simple network) price is %2.3f." % nn_simple_price)
print("The Deep Hedging (with recurrent network) price is %2.3f." % nn_recurrent_price)

In [0]:
#@title <font color='Blue'>**Results: Black-Scholes PnL vs Deep Hedging PnL**</font>

# Compute Black-Scholes delta for each sample path at each time t.
length_test_sample = len(S_test[0][:,0])
model_delta = np.zeros((length_test_sample, N+1))
func_delta = lambda i, j: report.get_model_delta(instrument, \
                s0=S_test[0][i,j], calculation_date=calculation_date + j)

for j in trange(N+1):
    model_delta[:,j]  = list(map(func_delta, range(length_test_sample), \
                            repeat(j,length_test_sample)))

# Compute Black-Scholes PnL (for a short position, i.e. the Bank sells
# a call option. The model delta from Quantlib is a long delta.
PnL_BS = (np.multiply(S_test[0][:,0], -model_delta[:,0]) - \
          np.abs(model_delta[:,0])*S_test[0][:,0]*epsilon)* \
          np.exp(risk_free/num_days_in_a_year)
for t in range(1, N):
  PnL_BS += np.multiply(S_test[0][:,t], -model_delta[:,t] + model_delta[:,t-1]) - \
               np.abs(model_delta[:,t] -model_delta[:,t-1])*S_test[0][:,t]*epsilon
  PnL_BS = PnL_BS*np.exp(risk_free/num_days_in_a_year)

PnL_BS += (np.multiply(S_test[0][:,N],model_delta[:,N-1]) + option_payoff_test[0] - \
              np.abs(model_delta[:,N-1])*S_test[0][:,N]*epsilon)

# Compute deep hedging PnL.
[PnL_Deep_Hedge, certainty_equiv] = model_simple.predict(xtest,batch_size=1000,verbose=0)

# Plot Black-Scholes PnL and Deep Hedging PnL (with BS_price charged on both).
fig_PnL = plt.figure(dpi= 125, facecolor='w')
fig_PnL.suptitle("Black-Scholes PnL vs Deep Hedging PnL \n", \
      fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_title("Simple Network Structure with epsilon = " + str(epsilon), \
      fontsize=8)
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist((PnL_BS+BS_price, reshape_1D(PnL_Deep_Hedge+BS_price)), \
         bins=30, label=["Black-Scholes PnL", "Deep Hedging PnL"])
ax.legend()
plt.show()

In [0]:
#@title <font color='Blue'>**Results: Black-Scholes Delta vs Deep Hedging Delta.**</font>
for days_from_today in (1,15,29):
  # x-axis: [Min_S, 10th percentile, ..., 90th percentile,  Max_S]
  S_range = np.sort(S_test[0][:,days_from_today])
  idx2select = np.append(np.arange(0, len(S_range), \
          int(len(S_range)/100)), len(S_range)-1)

  # Compute NN delta.
  submodel = Model(model_simple.input, model_simple.get_layer("dense_" + str(d+1) \
                + "_" + str(days_from_today)).output)
  nn_delta = submodel.predict(xtest, batch_size=1000,verbose=0)

  tmp_stack = np.stack((S_test[0][:,days_from_today],reshape_1D(nn_delta)),axis=1)
  tmp_stack = tmp_stack[tmp_stack[:,0].argsort()]
  tmp_stack = tmp_stack[idx2select]

  S_range = tmp_stack[:,0]
  nn_delta = tmp_stack[:,1]

  # Compute Black-Scholes delta for S_range.
  model_delta = np.zeros(S_range.shape)
  for i in range(len(S_range)):
    model_delta[i] = report.get_model_delta(instrument, s0= S_range[i], \
            calculation_date=calculation_date + days_from_today)
    
  # Create a plot of Black-Scholes delta against deep hedging delta.
  fig_delta = plt.figure(dpi= 125, facecolor='w')
  fig_delta.suptitle("Black-Scholes Delta vs Deep Hedging Delta \n", \
        fontweight="bold")
  ax_delta = fig_delta.add_subplot()
  ax_delta.set_title("Simple Network Structure with " + \
              "t=" + str(days_from_today) + ", " + \
                "epsilon=" + str(epsilon), \
                fontsize=8)
  ax_delta.set_xlabel("Price of the Underlying Asset")
  ax_delta.set_ylabel("Delta")
  ax_delta.plot(S_range, model_delta, label="Black-Scholes Delta")
  ax_delta.scatter(S_range,nn_delta, c="red", s=2, label="Deep Hedging Delta")
  ax_delta.legend()
  plt.show()

In [0]:
#@title <font color='Blue'>**Results: Simple vs Recurrent Network**</font>

# Compute deep hedging PnL for simple network.
try:
  tmp_model_str = model_data_dir + \
            "model_" + "simple" + "_"  + str(epsilon) + ".h5"
  model_simple = tf.keras.models.load_model(tmp_model_str )	
  [PnL_Deep_Hedge_simple, certainty_equiv] = \
            model_simple.predict(xtest,batch_size=1000,verbose=0)
except:
  print("File does not exist! You need to first run the simple network.")

# Compute deep hedging PnL for recurrent network.
try:
  tmp_model_str = model_data_dir + \
            "model_" + "recurrent" + "_"  + str(epsilon) + ".h5"
  model_recurrent = tf.keras.models.load_model(tmp_model_str )	
  [PnL_Deep_Hedge_recurrent, certainty_equiv] = \
            model_recurrent.predict(xtest,batch_size=1000,verbose=0)
except:
  print("File does not exist! You need to first run the recurrent network.")

# Plot Simple Network PnL vs Recurrent Network PnL (with BS_price charged on both).
print("\n")
fig_nn = plt.figure(dpi= 125, facecolor='w')
fig_nn.suptitle("Simple Network PnL vs Recurrent Network PnL \n ", \
      fontweight="bold")
ax = fig_nn.add_subplot()
ax.set_title("epsilon = " + str(epsilon), fontsize=8)
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist((reshape_1D(PnL_Deep_Hedge_simple+BS_price), \
          reshape_1D(PnL_Deep_Hedge_recurrent+BS_price)), \
         bins=30, label=["Simple Network PnL", "Recurrent Network PnL"])
ax.legend()
plt.show()

In [0]:
#@title <font color='Blue'>**Results: Black-Scholes Asymtotic**</font>
%autoreload 2
flag_load_BS_asym = True

# Plot Black-Scholes model price asymptotics.
# Aim to verify our results against Figure 10 in Buehler et al (2019, QF).
strategy_type = "simple"
range_epsilon = np.power(2.0,np.arange(-10,0,1)-5)
bs_price_asym = np.zeros_like(range_epsilon)

# For each epsilon, compute the deep-hedging (using simple network) price.
i = 0
for epsilon_asym in range_epsilon:
	optimizer = tfa.optimizers.LazyAdam(learning_rate=lr)
 
	model_asym = Deep_Hedging(N=N, d=d+2, m=m, risk_free=risk_free, \
							maturity=maturity).model( \
							strategy_type=strategy_type, epsilon = epsilon_asym, \
							loss_type=loss_type, loss_param = loss_param)
	model_asym.compile(optimizer=optimizer)

	best_model_file = model_data_dir + \
		"model_" + strategy_type + "_"  + str(epsilon_asym) + ".h5"

	# Determine stopping criteria for this part.
	early_stopping = EarlyStopping(monitor="loss", \
						patience=20, min_delta=1e-4, restore_best_weights=True)
	reduce_lr = ReduceLROnPlateau(monitor="loss", min_lr = 1e-4, \
						factor=0.95, patience=5, min_delta=1e-3, verbose=0)
	model_checkpoint = ModelCheckpoint(best_model_file, \
						monitor="loss", save_best_only=True, \
						save_weights_only=False, verbose=0)
	callbacks = [early_stopping, reduce_lr, model_checkpoint, TqdmCallback()]

	if not flag_load_BS_asym:
		print("Running Simple Network with epsilon equals %2.6f.\n" % epsilon_asym)
		model_asym_fit = model_asym.fit(x=xtrain, batch_size=batch_size, epochs=epochs, \
					validation_data=(xtest, np.empty(0)), callbacks=callbacks, \
					verbose=0)
		model_asym.save(best_model_file)
	else:
		model_asym_fit = model_asym.load_weights(best_model_file)
	
	bs_price_asym[i] = model_asym.evaluate(xtest, \
												batch_size=batch_size, verbose=0)
	print("The deep-hedging price is %2.3f.\n" % bs_price_asym[i])
		
	i += 1

clear_output()

# Create a plot of Black-Scholes Asymptotic - Figure 10 of Buehler et al (2019).
fig_asym = plt.figure(dpi= 125, facecolor='w')
fig_asym.suptitle("Black-Scholes Asymptotic \n", \
      fontweight="bold")
ax_asym = fig_asym.add_subplot()
ax_asym.set_xlabel("log(epsilon)")
ax_asym.set_ylabel("log(DH Price - BS Price)")

x = np.log(range_epsilon)
y = np.log(bs_price_asym - BS_price)

# Create trend (OLS) line.
[slope, const] = np.polyfit(x, y, 1)
ols_func = np.poly1d([slope, const])

ax_asym.set_title("Deep-Hedging Price (Simple Network) with different epsilons.", \
                    fontsize=8)
ax_asym.plot(x,ols_func(x),"r--", c="red", linewidth=0.8, alpha=0.5)
ax_asym.scatter(x, y, c="blue", s=2, label="log(DH Price - BS Price)")
ax_asym.legend()

props = dict(boxstyle='round', facecolor='wheat', alpha=0.1)
textstr = "y = " + "%.2f" % slope + "x + " + "%.2f" % const
ax_asym.text(0.1, 0.6, textstr, transform=ax_asym.transAxes, fontsize=11,
        verticalalignment='top', bbox=props)
plt.show()
